# Evaluate 
$ f(x, y) = 2x^2 - x^2y^2 + 3 $

$ g_0 = x ⋅ x $ 

$ g_1 = x ⋅ x $ 

$ g_2 = y ⋅ y $

$ g_3 = g_0 ⋅ 2 $

$ g_4 = g_1 ⋅ g_2 $

$ g_5 = g_2 - g_4 $

$ g_6 = g_5 + 3 $

---

$ c_0 = a_0 ⋅ b_0 $

$ c_1 = a_1 ⋅ b_1 $

$ c_2 = a_2 ⋅ b_2 $

$ c_3 = a_3 ⋅ 2 $

$ c_4 = a_4 ⋅ b_4 $

$ c_5 = a_5 - b_5 $

$ c_6 = a_6 + 3 $

---

$ x = a_0 = b_0 = a_1 = b_1 $

$ y = a_2 = b_2 $

$ g_0 = a_3 = c_0 $

$ g_1 = a_4 = c_1 $

$ g_2 = b_4 = c_2 $

$ g_3 = a_5 = c_3 $

$ g_4 = b_5 = c_4 $

$ g_5 = a_6 = c_5 $

$ g_6 = c_6 $

### Gate constrain equation

$ g_i = q_{Li}⋅a_i+q_{Ri}⋅b_i+q_{Mi}⋅(a_i⋅b_i)+q_{Ci}+q_{Oi}⋅c_i=0 $

$ g_0 = 0⋅a_0 + 0⋅b_0 + 1⋅(a_0⋅b_0) + 0 + (-1)⋅c_0 = 0 $

$ g_1 = 0⋅a_1 + 0⋅b_1 + 1⋅(a_1⋅b_1) + 0 + (-1)⋅c_1 = 0 $

$ g_2 = 0⋅a_2 + 0⋅b_2 + 1⋅(a_2⋅b_2) + 0 + (-1)⋅c_2 = 0 $

$ g_3 = 2⋅a_3 + 0⋅b_3 + 0⋅(a_3⋅b_3) + 0 + (-1)⋅c_3 = 0 $

$ g_4 = 0⋅a_4 + 0⋅b_4 + 1⋅(a_4⋅b_4) + 0 + (-1)⋅c_4 = 0 $

$ g_5 = 1⋅a_5 + (-1)⋅b_5 + 0⋅(a_5⋅b_5) + 0 + (-1)⋅c_5 = 0 $

$ g_6 = 1⋅a_6 + 0⋅b_6 + 0⋅(a_6⋅b_6) + 3 + (-1)⋅c_6 = 0 $

**selector vectors**

$ g_i = [q_{Li}, q_{Ri}, q_{Mi}, q_{Ci}, q_{Oi}] $

$ g_0 = [0, 0, 1, 0, -1] $

$ g_1 = [0, 0, 1, 0, -1] $

$ g_2 = [0, 0, 1, 0, -1] $

$ g_3 = [2, 0, 0, 0, -1] $

$ g_4 = [0, 0, 1, 0, -1] $

$ g_5 = [1, -1, 0, 0, -1] $

$ g_6 = [1, 0, 0, 3, -1] $

**let x = 2 y = 3**


| Vectors| a | b | c | qLi | qRi | qMi | qCi | qOi |
| --- | --- | --- | --- |--- | --- | --- | --- | --- |
| gate0 | 2 | 2 | 4 | 0 | 0 | 1 | 0 | -1 |
| gate1 | 2 | 2 | 4 | 0 | 0 | 1 | 0 | -1 |
| gate2 | 3 | 3 | 9 | 0 | 0 | 1 | 0 | -1 |
| gate3 | 4 | 0 | 8 | 2 | 0 | 0 | 0 | -1 |
| gate4 | 4 | 9 | 36 | 0 | 0 | 1 | 0 | -1 |
| gate5 | 8 | 36 | -28 | 1 | -1 | 0 | 0 | -1 |
| gate6 | -28 | 3 | -25 | 1 | 0 | 0 | 3 | -1 |

# Setup

In [1]:
import galois
import numpy as np
from utils import generator1, generator2, curve_order, normalize, validate_point

G1 = generator1()
G2 = generator2()


class SRS:
    def __init__(self, tau, n = 2):
        self.tau = tau
        self.tau1 = [G1 * int(tau)**i for i in range(0, n + 7)]
        self.tau2 = G2 * int(tau)

    def __str__(self):
        s = f"tau: {self.tau}\n"
        s += "".join([f"[tau^{i}]G1: {str(normalize(point))}\n" for i, point in enumerate(self.tau1)])
        s += f"[tau]G2: {str(normalize(self.tau2))}\n"
        return s

def new_call(self, at, **kwargs):
    if isinstance(at, SRS):
        coeffs = self.coeffs[::-1]
        result = at.tau1[0] * coeffs[0]
        for i in range(1, len(coeffs)):
            result += at.tau1[i] * coeffs[i]
        return result

    return galois.Poly.original_call(self, at, **kwargs)

galois.Poly.original_call = galois.Poly.__call__
galois.Poly.__call__ = new_call

In [2]:
x = 2
y = 3

# to switch between encrypted (ECC) and unencrypted mode (prime field p=241)
encrypted = True

# 2x^2 - x^2y^2 + 3
out = 2*x**2 - x**2*y**2 + 3
print(f"out = {out}")

# We have 7 gates, next power of 2 is 8
n = 7
n = 2**int(np.ceil(np.log2(n)))
assert n & n - 1 == 0, "n must be a power of 2"

# Prime field p
p = 241 if not encrypted else curve_order
# p = 241
Fp = galois.GF(p)

# Find primitive root of unity
omega = Fp.primitive_root_of_unity(n)
assert omega**(n) == 1, f"omega (ω) {omega} is not a root of unity"

roots = Fp([omega**i for i in range(n)])
print(f"roots = {roots}")

out = -25
roots = [                                                                            1
 19540430494807482326159819597004422086093766032135589407132600596362845576832
 21888242871839275217838484774961031246007050428528088939761107053157389710902
 13274704216607947843011480449124596415239537050559949017414504948711435969894
 21888242871839275222246405745257275088548364400416034343698204186575808495616
  2347812377031792896086586148252853002454598368280444936565603590212962918785
                    4407920970296243842541313971887945403937097133418418784715
  8613538655231327379234925296132678673308827349856085326283699237864372525723]


## Witness & Gates

In [3]:
def pad_array(a, n):
    return a + [0]*(n - len(a))

# witness vectors
a = [2, 2, 3, 4, 4, 8, -28]
b = [2, 2, 3, 0, 9, 36, 3]
c = [4, 4, 9, 8, 36, -28, -25]

# gate vectors
ql = [0, 0, 0, 2, 0, 1, 1]
qr = [0, 0, 0, 0, 0, -1, 0]
qm = [1, 1, 1, 1, 1, 0, 0]
qc = [0, 0, 0, 0, 0, 0, 3]
qo = [-1, -1, -1, -1, -1, -1, -1]
qpi = [0, 0, 0, 0, 0, 0, 0]

# pad vectors to length n
a = pad_array(a, n)
b = pad_array(b, n)
c = pad_array(c, n)
ql = pad_array(ql, n)
qr = pad_array(qr, n)
qm = pad_array(qm, n)
qc = pad_array(qc, n)
qo = pad_array(qo, n)
qpi = pad_array(qpi, n)

print(f"a = {a}")
print(f"b = {b}")
print(f"c = {c}")
print(f"ql = {ql}")
print(f"qr = {qr}")
print(f"qm = {qm}")
print(f"qc = {qc}")
print(f"qo = {qo}")
print(f"qpi = {qpi}")

a = [2, 2, 3, 4, 4, 8, -28, 0]
b = [2, 2, 3, 0, 9, 36, 3, 0]
c = [4, 4, 9, 8, 36, -28, -25, 0]
ql = [0, 0, 0, 2, 0, 1, 1, 0]
qr = [0, 0, 0, 0, 0, -1, 0, 0]
qm = [1, 1, 1, 1, 1, 0, 0, 0]
qc = [0, 0, 0, 0, 0, 0, 3, 0]
qo = [-1, -1, -1, -1, -1, -1, -1, 0]
qpi = [0, 0, 0, 0, 0, 0, 0, 0]


## Permutations

In [4]:
def print_sigma(sigma, a, b, c, r):
    group_size = len(sigma) // 3
    padding = 6

    print(f"{' w'} | {'value':{padding}} | {'i':{padding}} | {'sigma(i)':{padding}}")

    for i in range(0, group_size):
        print(f"a{i} | {a[i]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

    print(f"-- | {'--':{padding}} | {'--':{padding}} | {'--':{padding}}")

    for i in range(group_size, 2 * group_size):
        print(f"b{i - group_size} | {b[i - group_size]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

    print(f"-- | {'--':{padding}} | {'--':{padding}} | {'--':{padding}}")

    for i in range(2 * group_size, 3 * group_size):
        print(f"c{i - 2 * group_size} | {c[i - 2 * group_size]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

ai = range(0, n)
bi = range(n, 2*n)
ci = range(2*n, 3*n)

sigma = {
    ai[0]: ai[0], ai[1]: ai[1], ai[2]: ai[2], ai[3]: ci[0], ai[4]: ci[1], ai[5]: ci[3], ai[6]: ci[5], ai[7]: ai[7],
    bi[0]: bi[0], bi[1]: bi[1], bi[2]: bi[2], bi[3]: bi[3], bi[4]: ci[2], bi[5]: ci[4], bi[6]: bi[6], bi[7]: bi[7],
    ci[0]: ai[3], ci[1]: ai[4], ci[2]: bi[4], ci[3]: ai[5], ci[4]: bi[5], ci[5]: ai[6], ci[6]: ci[6], ci[7]: ci[7],
}

k1 = 2
k2 = 4
c1_roots = roots
c2_roots = roots * k1
c3_roots = roots * k2

c_roots = np.concatenate((c1_roots, c2_roots, c3_roots))

check = set()
for r in c_roots:
    assert not int(r) in check, f"Duplicate root {r} in {c_roots}"
    check.add(int(r))

sigma1 = Fp([c_roots[sigma[i]] for i in range(0, n)])
sigma2 = Fp([c_roots[sigma[i + n]] for i in range(0, n)])
sigma3 = Fp([c_roots[sigma[i + 2 * n]] for i in range(0, n)])

print_sigma(sigma, a, b, c, c_roots)

print("\n\n--- Cosest ---")
print(f"c0 = {c1_roots}")
print(f"c1 = {c2_roots}")
print(f"c2 = {c3_roots}")

print("\n\n--- Sigma ---")
print(f"sigma1 = {sigma1}")
print(f"sigma2 = {sigma2}")
print(f"sigma3 = {sigma3}")

 w | value  | i      | sigma(i)
a0 |      2 |      1 |      1
a1 |      2 | 19540430494807482326159819597004422086093766032135589407132600596362845576832 | 19540430494807482326159819597004422086093766032135589407132600596362845576832
a2 |      3 | 21888242871839275217838484774961031246007050428528088939761107053157389710902 | 21888242871839275217838484774961031246007050428528088939761107053157389710902
a3 |      4 | 13274704216607947843011480449124596415239537050559949017414504948711435969894 |      4
a4 |      4 | 21888242871839275222246405745257275088548364400416034343698204186575808495616 | 12496993363712103637900061152245863078729970927294254597435789825723956820477
a5 |      8 | 2347812377031792896086586148252853002454598368280444936565603590212962918785 | 9322331122753240927553110305983835483861419401407727382261611421694126888342
a6 |    -28 | 4407920970296243842541313971887945403937097133418418784715 | 9391249508127171584346344593011412009818393473121779746262414360851851675140

## Gate Polynomials

In [5]:
def to_galois_array(vector, field):
    # normalize to positive values
    a = [x % field.order for x in vector]
    return field(a)

def to_poly(x, v, field):
    assert len(x) == len(v)
    y = to_galois_array(v, field) if type(v) == list else v
    return galois.lagrange_poly(x, y)

QL = to_poly(roots, ql, Fp)
QR = to_poly(roots, qr, Fp)
QM = to_poly(roots, qm, Fp)
QC = to_poly(roots, qc, Fp)
QO = to_poly(roots, qo, Fp)
QPI = to_poly(roots, qpi, Fp)

print("--- Gate Polynomials ---")
print(f"QL = {QL}")
print(f"QR = {QR}")
print(f"QM = {QM}")
print(f"QC = {QC}")
print(f"QO = {QO}")
print(f"QPI = {QPI}")

--- Gate Polynomials ---
QL = 3612152601280961073314683502099786209434373305161036046916463827631781705411x^7 + 16416182153879456417235794430229986796728937546798018933265790281609158719802x^6 + 5961799955605786503393330439980659875369380680279914842076475412146455611834x^5 + 5472060717959818805561601436314318772137091100104008585924551046643952123904x^4 + 1859908116678857733348898176788593523338046287914958889992361502366775114672x^3 + 10944121435919637610572212751341607063956517953722023996356964951610601899719x^2 + 10454382198273669912189493626388235480406564127060124564712903444430796063700x + 10944121435919637611123202872628637544274182200208017171849102093287904247809
QR = 2442553811850935290769977449625552760761720754016948675891575074545355697104x^7 + 13680151794899547014454993712072827410660391996746014640303514758287182657850x^6 + 7131398745035812285938036492454893324042033231424002213101364165232881620141x^5 + 19152212512859365819465605027100115702479818850364030050735928663

## Permutation Polynomials

In [6]:
S1 = to_poly(roots, sigma1, Fp)
S2 = to_poly(roots, sigma2, Fp)
S3 = to_poly(roots, sigma3, Fp)

I1 = to_poly(roots, c1_roots, Fp)
I2 = to_poly(roots, c2_roots, Fp)
I3 = to_poly(roots, c3_roots, Fp)

padding = 3
for i in range(0, len(roots)):
    s  = f"i = {i:{padding}} --> {roots[i]:{padding}} "
    s += f"   I1({roots[i]:{padding}}) = {I1(roots[i]):{padding}} "
    s += f"   I2({roots[i]:{padding}}) = {I2(roots[i]):{padding}} "
    s += f"   I3({roots[i]:{padding}}) = {I3(roots[i]):{padding}} "
    s += f"   S1({roots[i]:{padding}}) = {S1(roots[i]):{padding}} "
    s += f"   S2({roots[i]:{padding}}) = {S2(roots[i]):{padding}} "
    s += f"   S3({roots[i]:{padding}}) = {S3(roots[i]):{padding}} "
    print(s)

    assert I1(roots[i]) == roots[i], f"I1({roots[i]}) != {roots[i]}"
    assert I2(roots[i]) == k1 * roots[i], f"I2({roots[i]}) != {k1 * roots[i]}"
    assert I3(roots[i]) == k2 * roots[i], f"I3({roots[i]}) != {k2 * roots[i]}"

    assert S1(roots[i]) == sigma1[i], f"S1({roots[i]}) != {sigma1[i]}"
    assert S2(roots[i]) == sigma2[i], f"S2({roots[i]}) != {sigma2[i]}"
    assert S3(roots[i]) == sigma3[i], f"S3({roots[i]}) != {sigma3[i]}"

i =   0 -->   1    I1(  1) =   1    I2(  1) =   2    I3(  1) =   4    S1(  1) =   1    S2(  1) =   2    S3(  1) = 13274704216607947843011480449124596415239537050559949017414504948711435969894 
i =   1 --> 19540430494807482326159819597004422086093766032135589407132600596362845576832    I1(19540430494807482326159819597004422086093766032135589407132600596362845576832) = 19540430494807482326159819597004422086093766032135589407132600596362845576832    I2(19540430494807482326159819597004422086093766032135589407132600596362845576832) = 17192618117775689430073233448751569083639167663855144470566997006149882658047    I3(19540430494807482326159819597004422086093766032135589407132600596362845576832) = 12496993363712103637900061152245863078729970927294254597435789825723956820477    S1(19540430494807482326159819597004422086093766032135589407132600596362845576832) = 19540430494807482326159819597004422086093766032135589407132600596362845576832    S2(195404304948074823261598195970044220860937660321355

## CRS Construction

In [7]:
def generate_tau(encrypted=False):
    return SRS(Fp.Random(), n) if encrypted else Fp.Random()

tau = generate_tau(encrypted=encrypted)
print(f"--- Tau ---")
print(tau)

--- Tau ---
tau: 9656377472822928631572348973013539797546162235727922534767800958186450015691
[tau^0]G1: (1, 2)
[tau^1]G1: (3157155566928353448294608396861862562408722407632382857881496235596007045323, 8740325234551046900714484292781223002358198867067910477236498551241785912525)
[tau^2]G1: (1395557350822679137076921302989016615499078588490143183692656918924483289776, 8119103352802068587783068355254172235032500428093109503945973474427212133642)
[tau^3]G1: (2338041747085731839180364255766151431109909514471096514691427960718170998264, 7970441833434412847996968017389921346815799750753795765792442712813168318374)
[tau^4]G1: (3119619251334572232655763614593200909477975126394277395359520624108399817385, 19694263922213630171549210464061933513105526339696026948595480102733125021313)
[tau^5]G1: (17552043593335682867402231774008692257399435643874171043139202588894131005536, 14903242401450927336708232574592962574174507588826751407535130473879067114477)
[tau^6]G1: (664466491443580855245129407477670

# Prover

In [8]:
def to_vanishing_poly(roots, field):
    # Z^n - 1 = (Z - 1)(Z - w)(Z - w^2)...(Z - w^(n-1))
    return galois.Poly.Degrees([len(roots), 0], coeffs=[1, -1], field=field)

Zh = to_vanishing_poly(roots, Fp)
for x in roots:
    assert Zh(x) == 0

print("--- Vanishing Polynomial ---")
print(f"Zh = {Zh}")

--- Vanishing Polynomial ---
Zh = x^8 + 21888242871839275222246405745257275088548364400416034343698204186575808495616


## Round 1

In [9]:
random_b = [Fp.Random() for i in range(0, 9)]

bA = galois.Poly(random_b[:2], field=Fp)
bB = galois.Poly(random_b[2:4], field=Fp)
bC = galois.Poly(random_b[4:6], field=Fp)

_A = to_poly(roots, a, Fp)
_B = to_poly(roots, b, Fp)
_C = to_poly(roots, c, Fp)

A = _A + bA*Zh
B = _B + bB*Zh
C = _C + bC*Zh

# gate constraints polynomial
# g(x) = a(x)*ql(x) + b(x)*qr(x) + a(x)*b(x)*qm(x) + qc(x) + c(x)*qo(x)
G = A*QL + B*QR + A*B*QM + QC + C*QO + QPI

print("--- Gate Constraints Polynomial ---")
print(f"G = {G}")
for i in range(0, len(roots)):
    print(f"gate #{i} G({roots[i]}) = {G(roots[i])} --> {'OK' if G(roots[i]) == 0 else 'FAIL'}")
    assert G(roots[i]) == 0, f"G({roots[i]}) != 0"

assert G % Zh == 0, f"G(x) % Zh(x) != 0"

padding = 3
for i in range(0, len(roots)):
    s = f"i = {i:{padding}} --> {roots[i]:{padding}} "
    s += f"   A({roots[i]:{padding}}) = {A(roots[i]):{padding}} "
    s += f"   B({roots[i]:{padding}}) = {B(roots[i]):{padding}} "
    s += f"   C({roots[i]:{padding}}) = {C(roots[i]):{padding}} "
    print(s)

round1 = [A(tau), B(tau), C(tau)]
print("\n\n--- Round 1 ---")
print(f"Round 1 = {round1}")
print("TODO: [A], [B], [C]")

--- Gate Constraints Polynomial ---
G = 4831071989495668124400420300230689388265359070583392849257340002818958764323x^25 + 9124269981847212748522888464926299466720414851816252503776456480526656641678x^24 + 3979565848294808056685482369406531608076688347591077118153322710259310798695x^23 + 1427164821782564581891778179991157365145848740119846010351561181869856167309x^22 + 17737779664218290858148883312287478580326289962761931810730608280189789425634x^21 + 17705471781881916800788380437292637305310709608035364735443075491672411551267x^20 + 18310263178547475356928614142486005263480433426943498465197010513600141671485x^19 + 5524300083868629040956394742635647047862765631073510978135253635205909443686x^18 + 7223293997853695576182232651187446646371337859790168720809556248669695352495x^17 + 2113645560400201230597582576795720463916005684350703750899917615909343519710x^16 + 14839012916932444822647481640798002518571612985422047843354296166863476609580x^15 + 1084153252253781241206240019008339667293216

## Round 2

In [10]:
import sha3

def numbers_to_hash(numbers, field) -> int:
    """Hash a number."""
    engine = sha3.keccak_256()
    for number in numbers:
        if isinstance(number, tuple):
            x, y, z = number
            engine.update(bytes(hex(int(x)), 'utf-8'))
            engine.update(bytes(hex(int(y)), 'utf-8'))
            engine.update(bytes(hex(int(z)), 'utf-8'))
        else:
            engine.update(bytes(hex(int(number)), 'utf-8'))
    return field(int(engine.hexdigest(), 16) % field.order)

# TODO: calculate beta, gamma from [A], [B], [C]
beta = numbers_to_hash(round1 + [0], Fp)
gamma = numbers_to_hash(round1 + [1], Fp)

_F = (A + I1 * beta + gamma) * (B + I2 * beta + gamma) * (C + I3 * beta + gamma)
_G = (A + S1 * beta + gamma) * (B + S2 * beta + gamma) * (C + S3 * beta + gamma)

acc_eval = [Fp(1)]
for i in range(0, n):
    acc_eval.append(
        acc_eval[-1] * (_F(roots[i]) / _G(roots[i]))
    )
assert acc_eval.pop() == Fp(1)
ACC = galois.lagrange_poly(roots, Fp(acc_eval))
print("\n\n--- Accumulator Polynomial ---")
print(f"ACC(x) = {ACC}")

bZ = galois.Poly(random_b[6:9], field=Fp)
print(f"bZ = {bZ}")
Z = bZ * Zh + ACC

print("\n\n--- Z Polynomial ---")
print(f"Z(x) = {Z}")

for r in roots:
    print(f"Z({r}) = {Z(r)}")

assert Z(roots[0]) == 1
assert Z(roots[-1]) == 1

round2 = [Z(tau)]
print("\n\n--- Round 2 ---")
print(f"Round 2 = {round2}")
print("TODO: [Z]")



--- Accumulator Polynomial ---
ACC(x) = 19988255256687092258754524423934934028348474745481243870676139197202943180581x^7 + 19306030441079121686236703698738099791487569219954178054851074168929463028607x^6 + 14807912109742452741988853645717746473276301754373153960191514828630202983962x^5 + 4949122303403767482374880327759029188073765275034487039178449124843868489698x^4 + 11617233688089035317699289309131214070635953837019397209949128729456523447057x^3 + 1506112429863398459064238269712728546106423101574810024972867596229852479615x^2 + 15950479964782317121859327840806035107451766000820288568824368278647873736882x + 21316068165549191043254211210486588237361568067822612989847479008938315131684
bZ = 8522255782870514862203240631704832616064276004402929961004479582086040066479x^2 + 8723626377056050052395636946835678540690249314598168640517458996852130597633x + 3524670563949247536393326933937199583339093867836347329745487879414191206150


--- Z Polynomial ---
Z(x) = 85222557828705148622032406317

# Round 3

In [11]:
def shift_poly(poly: galois.Poly, omega: Fp):
    coeffs = poly.coeffs[::-1]
    coeffs = [c * omega**i for i, c in enumerate(coeffs)]
    return galois.Poly(coeffs[::-1], field=poly.field)

alpha = numbers_to_hash(round1 + round2, Fp)

Zomega = shift_poly(Z, omega=omega)
for r in roots:
    print(f"Z({r:3}) = {Z(r):3} -> Zω({r:3}) = {Zomega(r):3}")

print("\n\n--- Zω Polynomial ---")
print(f"Z(ωx) = {Zomega}")

L1 = galois.lagrange_poly(roots, Fp([1] + [Fp(0)] * (n - 1)))

print("\n\n--- L1 Polynomial ---")
print(f"L1(x) = {L1}")
for i, r in enumerate(roots):
    print(f"L1({r}) = {L1(r)}")
    assert L1(r) == (Fp(1) if i == 0 else Fp(0))

T0 = G
assert T0 % Zh == 0, f"T0(x) % Zh(x) != 0"

T1 = (_F * Z - _G * Zomega) * alpha
assert T1 % Zh == 0, f"T1(x) % Zh(x) != 0"

T2 = (Z - galois.Poly([1], field=Fp)) * L1 * alpha**2
assert T2 % Zh == 0, f"T2(x) % Zh(x) != 0"

T = (T0 + T1 + T2)
assert T % Zh == 0, f"T(x) % Zh(x) != 0"

for r in roots:
    assert T(r) == 0, f"T({r}) != 0"

T = T // Zh

print("\n\n--- T Polynomial ---")
print(f"T(x) = {T}")

t_coeffs = T.coeffs[::-1]

Tl = galois.Poly(t_coeffs[:n][::-1], field=Fp)
Tm = galois.Poly(t_coeffs[n:2*(n)][::-1], field=Fp)
Th = galois.Poly(t_coeffs[2*(n):][::-1], field=Fp)

X_n = galois.Poly.Degrees([n, 0], coeffs=[1, 0], field=Fp)
X_2n = galois.Poly.Degrees([2*(n), 0], coeffs=[1, 0], field=Fp)
# make sure that T was split correctly
# T = TL + X^n * TM + X^2n * TH
assert T == (Tl + X_n * Tm + X_2n * Th)
assert T.degree == 3 * n + 5

b10 = Fp.Random()
b11 = Fp.Random()

Tl = Tl + b10 * X_n
Tm = Tm - b10 + b11 * X_n
Th = Th - b11
assert T == (Tl + X_n * Tm + X_2n * Th)

print("\n\n--- T' ---")
print(f"Tl(x) = {Tl}")
print(f"Tm(x) = {Tm}")
print(f"Th(x) = {Th}")

round3 = [Tl(tau), Tm(tau), Th(tau)]
print("\n\n--- Round 3 ---")
print(f"Round 3 = {round3}")
print("TODO: [Tl], [Tm], [Th]")

Z(  1) =   1 -> Zω(  1) = 133531771604891812430017174108315399086944941874013099564845189766335937586
Z(19540430494807482326159819597004422086093766032135589407132600596362845576832) = 133531771604891812430017174108315399086944941874013099564845189766335937586 -> Zω(19540430494807482326159819597004422086093766032135589407132600596362845576832) = 9717040655394421084939191734909454577928033369649687565331348942450336970702
Z(21888242871839275217838484774961031246007050428528088939761107053157389710902) = 9717040655394421084939191734909454577928033369649687565331348942450336970702 -> Zω(21888242871839275217838484774961031246007050428528088939761107053157389710902) = 7272141887267371350855964398260786275422060862931341109024539139007368192160
Z(13274704216607947843011480449124596415239537050559949017414504948711435969894) = 7272141887267371350855964398260786275422060862931341109024539139007368192160 -> Zω(13274704216607947843011480449124596415239537050559949017414504948711435969894) = 6601

## Round 4

In [12]:
zeta = numbers_to_hash(round1 + round2 + round3, Fp)

a_zeta = A(zeta)
b_zeta = B(zeta)
c_zeta = C(zeta)
s1_zeta = S1(zeta)
s2_zeta = S2(zeta)
t_zeta = T(zeta)
z_omega_zeta = Zomega(zeta)

R = (QM * a_zeta * b_zeta +
    QL * a_zeta +
    QR * b_zeta +
    QO * c_zeta +
    QC)
R += (Z * 
    (a_zeta + beta * zeta + gamma) *
    (b_zeta + beta * zeta * k1 + gamma) *
    (c_zeta + beta * zeta * k2 + gamma) * alpha)
R -= (z_omega_zeta *
    (a_zeta + beta * s1_zeta + gamma) *
    (b_zeta + beta * s2_zeta + gamma) * 
    (c_zeta + beta * S3 + gamma) * alpha)
R += (Z - Fp(1)) * L1(zeta) * alpha**2
R -= Zh(zeta) * (Tl + zeta**n * Tm + zeta**(2*n) * Th)

print("\n\n--- R ---")
print(f"R(x) = {R}")

r_zeta = R(zeta)

round4 = [a_zeta, b_zeta, c_zeta, s1_zeta, s2_zeta, t_zeta, z_omega_zeta, r_zeta]
print("\n\n--- Round 4 ---")
print(f"Round 4 = {round4}")



--- R ---
R(x) = 10690064466764504555703120486878294626984399725855121749013137662560103265068x^13 + 13304195423721992662936433095243662391265230595975733817506236662619752547659x^12 + 2917263930061577243828333083196621867791825956387322258310017502367161490769x^11 + 13880972521127480329870224975003870233066496882443595013359707176847817013051x^10 + 2639491573889035872707982377438823681975614242414362418530868914758899037153x^9 + 11563387130533696256371790326332143852549956761535068615489114009276046415945x^8 + 13618346212725911996018080850272300495433592673411082660340035073572600464350x^7 + 3298618595018299565751729323653487351949183652105597257082751496573370989396x^6 + 3216100510736184930734066585566238972321880760816983488671411086283500663131x^5 + 10782130915906714283146678306145441032796532201885483662553627589060660386871x^4 + 9546524102021342459550953256759801415890061575198443559648056421618184518187x^3 + 113172642439906224304756398952013226860765162815827605304961153380494

# Round 5

In [13]:
v = numbers_to_hash(round1 + round2 + round3 + round4, Fp)

X_minus_zeta = galois.Poly([1, -zeta], field=Fp)
print(f"X - zeta = {X_minus_zeta}")

Wzeta = R + \
        (A - a_zeta) * v + \
        (B - b_zeta) * v**2 + \
        (C - c_zeta) * v**3 + \
        (S1 - s1_zeta) * v**4 + \
        (S2 - s2_zeta) * v**5

assert Wzeta % X_minus_zeta == 0, f"Wzeta(x) % X - zeta != 0"
Wzeta = Wzeta // X_minus_zeta

X_minus_omega_zeta = galois.Poly([1, -(omega*zeta)], field=Fp)
print(f"X - ω*zeta = {X_minus_omega_zeta}")

Womega_zeta = (Z - z_omega_zeta)
assert Womega_zeta % X_minus_omega_zeta == 0, f"Womega_zeta(x) % X - ω*zeta != 0"
Womega_zeta = Womega_zeta // X_minus_omega_zeta

round5 = [Wzeta(tau), Womega_zeta(tau)]
print("\n\n--- Round 5 ---")
print(f"Round 5 = {round5}")
print("TODO: [Wzeta], [Womega_zeta]")

u = numbers_to_hash(round1 + round2 + round3 + round4 + round5, Fp)

X - zeta = x + 8334800690363183807110796767717986348724275185254913793432552840961450500699
X - ω*zeta = x + 16296570596005947621414441055573410409478903937083970393986445738457278309651


--- Round 5 ---
Round 5 = [(15891720053803529713781849394841360020534984931592770806409279099705343558193, 5481365667603385545728308054577837855639790722379888121435817267057116594258, 13246000404950105076683009311649100055655826270058968723167450579016366479273), (5707712556157445581450999013264316499859648761518582882763546118686245322029, 3628596653025101720467397288126663257180935606138102923131968315770835142203, 19910306924183202299276206851027416738306027827691419983023085757229965887706)]
TODO: [Wzeta], [Womega_zeta]


# Verifier

In [14]:
qm_exp = QM(tau)
ql_exp = QL(tau)
qr_exp = QR(tau)
qo_exp = QO(tau)
qc_exp = QC(tau)
qpi_exp = QPI(tau)
z_exp = Z(tau)
s1_exp = S1(tau)
s2_exp = S2(tau)
s3_exp = S3(tau)
tl_exp = Tl(tau)
tm_exp = Tm(tau)
th_exp = Th(tau)

a_exp = A(tau)
b_exp = B(tau)
c_exp = C(tau)

w_zeta_exp = Wzeta(tau)
w_omega_zeta_exp = Womega_zeta(tau)

if encrypted:
    validate_point(qm_exp)
    validate_point(ql_exp)
    validate_point(qr_exp)
    validate_point(qo_exp)
    validate_point(qc_exp)
    validate_point(qpi_exp)
    validate_point(z_exp)
    validate_point(s1_exp)
    validate_point(s2_exp)
    validate_point(s3_exp)
    validate_point(tl_exp)
    validate_point(tm_exp)
    validate_point(th_exp)
    validate_point(a_exp)
    validate_point(b_exp)
    validate_point(c_exp)
    validate_point(w_zeta_exp)
    validate_point(w_omega_zeta_exp)

In [15]:
Zh_z = Zh(zeta)
L1_z = L1(zeta)

r0 = (- L1_z * alpha**2 -
    (a_zeta + beta * s1_zeta + gamma) *
    (b_zeta + beta * s2_zeta + gamma) *
    (c_zeta + gamma) * z_omega_zeta * alpha)

In [16]:
D_exp = (qm_exp * a_zeta * b_zeta +
        ql_exp * a_zeta +
        qr_exp * b_zeta +
        qo_exp * c_zeta +
        qc_exp)

D_exp += (z_exp * (
        (a_zeta + beta * zeta + gamma) *
        (b_zeta + beta * zeta * k1 + gamma) *
        (c_zeta + beta * zeta * k2 + gamma) * alpha
        + L1_z * alpha**2 + u))

D_exp -= (s3_exp *
        (a_zeta + beta * s1_zeta + gamma) *
        (b_zeta + beta * s2_zeta + gamma) * 
        alpha * beta * z_omega_zeta)

D_exp -= ((tl_exp + 
        tm_exp * zeta**n  +
        th_exp * zeta**(2*n)) *
        Zh_z)

In [17]:
F_exp = (D_exp + 
        a_exp * v +
        b_exp * v**2 +
        c_exp * v**3 +
        s1_exp * v**4 +
        s2_exp * v**5)

In [18]:
E_exp = (-r0 +
        v * a_zeta +
        v**2 * b_zeta +
        v**3 * c_zeta +
        v**4 * s1_zeta +
        v**5 * s2_zeta +
        u * z_omega_zeta)

if encrypted:
        E_exp = G1 * E_exp

In [19]:
e1 = w_zeta_exp + w_omega_zeta_exp * u
e2 = (w_zeta_exp * zeta + w_omega_zeta_exp * (u * zeta * omega) +
    F_exp + (E_exp * Fp(p-1)))

if encrypted:
    pairing1 = tau.tau2.pairing(e1)
    pairing2 = G2.pairing(e2)

    print(f"pairing1 = {pairing1}")
    print(f"pairing2 = {pairing2}")

    assert pairing1 == pairing2, f"pairing1 != pairing2"
else:
    print("\n\n--- e1, e2 ---")
    print(f"e1 = {e1 * tau}")
    print(f"e2 = {e2}")
    assert e1 * tau == e2

pairing1 = (19586301197894318323268773793930222598894522766725068380546407135945234838599, 9449599270956514171175456929356381233993779441356423745921541310251138005858, 4165377824276659661160470349923666131473903317908861944243167199667192390578, 6704847390703384775795635380252830305205719971178846745935909003984978042861, 18314512822641313543595855320608846614227931288438382483277477674622552371029, 8653103415087263841322131382939139982044403033946382326069880720140764730871, 8213770583784602287386060853654485200238098397146018953326192259310583464802, 7122699893722059642626009910507799476785358449280208015265663346976326585023, 15367420045694869541254659597493503492063922057704815567407949300354602481043, 1519448044203389851037625789171318526574198497884500236050061072153284967405, 20978673029725005876919449345283521432172305411804289936607159126124702971683, 4262319522514004311165301833343493356829752809161405712731774472948217724877)
pairing2 = (195863011978943183232687737939302225